## Word to word similarity of questions:

#### Importing the libraries:

In [1]:
import pandas as pd
import numpy as np
import time
import pandas_gbq
from google.cloud import bigquery
import os,os.path
from xmlrpc import client
import db_dtypes

In [2]:
# Estrablishing connection:
auth_json_path = "D:\iamneo.ai\dup-ques-w2w-iamneo\examly-events-26d60ca289a0.json"

os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.path.expanduser(auth_json_path)

In [3]:
client = bigquery.Client()

query = '''
SELECT
*,
(select qb_name from `examly-events.examly_warehouse_mysql_replica.question_banks` where qb_id = dup.qb_id) as qb_name
FROM `examly-events.schema_testing_views.questions_dup_view` dup
'''

#### Fetching all the data

In [4]:
whole_data = pandas_gbq.read_gbq(query, use_bqstorage_api=True,dialect="standard")

Downloading: 100%|██████████████████████████████████████████████████████| 9279104/9279104 [06:31<00:00, 23713.72rows/s]


In [5]:
# Number of question type present in the data
whole_data["question_type"].nunique()

19

# Data Insights:

In [6]:
# 
whole_data["question_type"].value_counts()

mcq_single_correct                 8567512
programming                         393877
descriptive_answer_writing          153574
fillup_single_correct               110533
essay_email_writing                  11656
file_upload_questions                 9401
descriptive_answer_writing_text       9172
mcq_multiple_correct                  8061
programming_file_based                7690
html_css_js                           4964
frontend_technology                   1042
project_question                       922
diagram                                389
video_answer_speaking                  189
classification                          74
cloud_question                          17
question_paper_upload                   15
block_programming                        9
                                         7
Name: question_type, dtype: int64

In [7]:
#whole_data["question_type"].value_counts().to_excel("q_type_contribution.xlsx")

In [7]:
#filtering the mcq_single_correct data as it has the maximum contribution.
mcq_single_correct_data = whole_data.iloc[np.where(whole_data["question_type"] == "mcq_single_correct")]

In [8]:
mcq_single_correct_data.head(1)

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
1,447a7ce9-476a-48ec-b5c4-40d495ae0087,0c45b707-2917-4a37-aaf1-3908d92f2048,"<p>In an LLG fault, the zero sequence and nega...",Easy,mcq_single_correct,original_question,5e2f4532-bf80-4a29-8599-16f91a09d478,2015798e-92bd-46f8-ae4a-8c5969744bfe,ac3f5f5b-635b-4a1b-834f-61b784e197b6,Engineering,Electrical,Power System,2019-10-15 05:48:03,ea3bf5eb-c116-420d-afa8-1585a2283c9d,NaT,NaT,cmrit977,"[{""text"":""<p>Series </p>"",""media"":""""},{""text""...","{""args"":[""<p>Parallel </p>""],""partial"":[]}",POWER SYSTEM ANALYSIS 1 15EE62


In [9]:
mcq_single_correct_data["subject"].unique()

array(['Engineering', 'Test Subject', 'Programming', 'bulk-imported',
       'COE', 'Nandha', 'Commerce', 'MCQ', 'C8-Maths', 'Aptitude',
       'End Semester', 'Soft Skills', 'Communication Skills',
       'Logical skills', 'ECE', 'History', 'Mechanical',
       'Environment and Ecology', 'Management', 'Essay', 'Social Justice',
       'Indian Economy', 'TEACHING METHODOLOGY - PART 1',
       'Java Full Stack Development', 'Art and Culture',
       'Domain Training', 'CSAT', 'C++', 'Polity', 'Finance and Accounts',
       'Personality', 'French', 'Geography', 'Technical',
       'Computer Applications', 'Kongu', 'Java',
       'International Relations', 'CSE',
       'Database Management System (DBMS)', 'Science and Technology',
       'Business Development and Sales', 'EEE', 'Linux', 'Data',
       'Class 6 - Maths', 'BCM_Basic Civil and Mechanical Engineering',
       'Civil', 'C6-Maths', None, 'Data Engineering (Python)',
       'Web Technology and its Applications (WT)',
       'Co

In [10]:
len(list(mcq_single_correct_data["subject"].unique()))

79

In [11]:
mcq_single_correct_data.shape

(8567512, 20)

In [12]:
mcq_single_correct_data["mcq_questions_options"].isna().sum()

26740

In [13]:
# mcq_questions_options with none values --> y
#y = mcq_single_correct_data.iloc[np.where(mcq_single_correct_data["mcq_questions_options"].isna() == True)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [35]:
#y["subject"].value_counts()

COE                      6200
Aptitude                  903
Programming               321
Engineering                50
VISTAS                     23
Test Subject               18
Data                        2
MCQ                         2
Computer Applications       1
Name: subject, dtype: int64

In [12]:
# Empty 
mcq_single_correct_data["mcq_questions_options"].isnull().sum()

26740

In [13]:
# Empty question rows
mcq_single_correct_data.iloc[np.where(mcq_single_correct_data["question_data"] == '<p></p>')].shape

(73140, 20)

In [14]:
# mcq_single_correct[question_data] having empty question --> x
x = mcq_single_correct_data.iloc[np.where(mcq_single_correct_data["question_data"] == '<p></p>')]

In [34]:
idx = [] # empty mcq rows whose question deletion or qb deletion data is absent.
for i in range(len(x["question_data"])):
    if x["q_deletedAt"].iloc[i] is pd.NaT:
        if x["qb_deletedAt"].iloc[i] is pd.NaT:
            if x["mcq_questions_options"].iloc[i] == '[]':
                idx.append(i)

In [35]:
len(idx)

1006

In [36]:
x.iloc[idx] # data with empty question data, empty question deleted data, empty qb deleted data and empty options data.
# mostly created before 2021, can be ignored.

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
578782,966b29e0-0ceb-4f1f-af59-02acf4f8a5e9,d5fc848b-b972-453b-b61d-6fa2a2f82f85,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-16 05:18:01,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},CTS Topic-Wise QP 16.10.2020
579019,28c95a3a-a8b8-4648-abcb-0de17cebeeb7,5b69c33e-eb69-49bf-b2cc-4cb01e890622,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-11-18 11:14:20,1fbf5ae7-e849-4545-b49f-d0e1e7f4ed6d,NaT,NaT,hicet902,[],{},19MT3203_ Unit _V_Part A_DAMS
590810,26e3ed46-4df3-4749-ba21-a42091a65fb5,8637bd1b-a2bc-46eb-a297-788a6704bcec,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-11-21 05:46:49,1fbf5ae7-e849-4545-b49f-d0e1e7f4ed6d,NaT,NaT,hicet902,[],{},19ME3201_MFT I_U3_PART A
591576,2083120c-99d6-455e-bc28-af82d0106d19,e514f182-9234-4440-b801-94c6549cf368,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-23 08:39:00,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},Time Speed Distance
607131,a0fa672a-a332-4ca8-a6da-1f1cdad2b176,16a2b618-f9f1-46f7-884b-f64ca795667e,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2018-09-27 11:00:54,511d98eb-361f-4bdb-8cc9-f0f62cb96edd,NaT,NaT,spark327,[],{},Synonyms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5538922,13d7e80a-7571-435b-a30a-5c6633bd4a40,54baae24-d0ab-478e-bdc4-18a601b430bb,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-17 03:20:47,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},TCS Ages Numbers Profit Ratio
5538963,48012fc2-8c4b-4c85-b267-8b0f0059b3d1,df50aafc-2464-4714-b5d6-08977f400da7,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-17 03:24:39,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},TCS Logical Reasoning II
5555617,b9c0f255-d591-4624-95b4-80bf44d74bc7,92a49bb0-002d-4269-8c22-a3844e83727e,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-17 03:35:20,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},TCS Permutation
5620802,fa0852cd-ee46-4ee9-9d8f-1c5c3c84f5dd,859871ab-7eaf-4bdb-b139-f2370f8b1ea4,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2019-12-31 10:13:24,f8feb13d-df22-43ec-acb8-31f873d9d104,NaT,NaT,sixphrase111,[],{},FLUID MECHANICS AND MACHINERY


In [20]:
x.iloc[idx].to_excel("Question_qDelete_qbDelete_options_empty_inMCQ.xlsx")

In [37]:
x = x.drop(index=x.iloc[idx].index)

In [38]:
x.shape

(72134, 20)

In [39]:
# removing all the questions from mcq_single correct data whose data is deleted or is empty:
mcq_single_correct_data = mcq_single_correct_data.drop(index=x.index)

In [40]:
mcq_single_correct_data.shape

(8495378, 20)

In [41]:
mcq_single_correct_data.isnull().sum()

qb_id                          0
q_id                           0
question_data                  0
manual_difficulty              0
question_type                  0
imported                       0
subject_id               2550649
topic_id                 2545212
sub_topic_id             2545214
subject                  2550649
topic                    2545212
sub_topic                2545214
createdAt                      0
school_id                  27918
q_deletedAt              8009579
qb_deletedAt             8176327
school_code                27918
mcq_questions_options      26698
mcq_questions_answer       26698
qb_name                        0
dtype: int64

In [42]:
idx_whole = [] # empty mcq rows whose question deletion or qb deletion data is absent.
for i in range(len(mcq_single_correct_data["question_data"])):
    if mcq_single_correct_data["q_deletedAt"].iloc[i] is pd.NaT:
        if mcq_single_correct_data["qb_deletedAt"].iloc[i] is pd.NaT:
            if mcq_single_correct_data["mcq_questions_options"].iloc[i] == '[]':
                idx_whole.append(i)

In [43]:
len(idx_whole)

2320

In [44]:
mcq_single_correct_data.iloc[idx_whole]

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
96862,419cd94b-ad18-4580-9dc0-b7c4bf825e11,18befd39-f8fb-479c-8365-b98044e3588e,<p>return 0; }</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:22:38,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 4 - technical
271223,daa576d4-93dc-4dad-b6b9-80ab2f26afab,521b9f3e-5071-46cb-9ab0-c68cd45bf722,<p>} }</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-09-05 09:04:05,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 5- Technical
375680,419cd94b-ad18-4580-9dc0-b7c4bf825e11,3d7713f4-be70-43c6-9a1a-589dc991a292,<p>}</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:22:39,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 4 - technical
469452,419cd94b-ad18-4580-9dc0-b7c4bf825e11,2a0f82e8-054f-4379-8aaf-a7a7f55f2bda,"<p>printf(""""Now the letter is"""");</p>",Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:22:38,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 4 - technical
578782,966b29e0-0ceb-4f1f-af59-02acf4f8a5e9,d5fc848b-b972-453b-b61d-6fa2a2f82f85,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-16 05:18:01,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},CTS Topic-Wise QP 16.10.2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8436065,a934c494-1226-4068-8500-3dacc1a0ee82,6490dad6-9e66-4c52-83ca-fdce4a86bf56,<p>What are the military wastes mentioned by E...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,6912740f-6977-4423-bfe1-ede3197c5796,VISTAS,SemExam,Unit 2,2020-12-10 10:15:17,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_18LENG21_DESC- BCOM AF
8456932,4d5534ab-4fd5-4f85-a5f7-0ca9fe32a803,f19eac0f-adde-4269-a57d-1e30921358b8,<p>Define the following terms:\na)Acceptable l...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-10 08:41:13,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_16CBAV54_DESC
8463344,e33ef61c-caad-409a-94a5-f4ebd4225190,de50656b-69ba-47f3-b78e-280ad5997df4,<p>Explain Types Of Licences and Ratings?- Air...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-10 06:48:04,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_16CBAV31_DESC
8492234,1604127b-b4c7-4247-8e79-070ec8d3d0be,5d4f898c-cb5c-40b5-b354-031f184fc731,<p>List out the general functions of aminoacid...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,4999dca1-12f6-4c0f-9e8a-bb75efda54a8,VISTAS,SemExam,Unit 3,2020-12-10 10:53:51,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_12CBBC31_DESC


In [45]:
mcq_single_correct_data = mcq_single_correct_data.drop(index=mcq_single_correct_data.iloc[idx_whole].index)

In [46]:
mcq_single_correct_data.shape

(8493058, 20)

In [47]:
idx_whole = [] # empty mcq rows whose question deletion or qb deletion data is absent.
for i in range(len(mcq_single_correct_data["question_data"])):
    if mcq_single_correct_data["q_deletedAt"].iloc[i] is pd.NaT:
        if mcq_single_correct_data["qb_deletedAt"].iloc[i] is pd.NaT:
            if mcq_single_correct_data["mcq_questions_options"].iloc[i] == '[]':
                idx_whole.append(i)

In [48]:
len(idx_whole)# hence no mcq rows whose question deletion or qb deletion data is absent.

0

In [36]:
# question_data = <p></p> &  mcq_questions_options = [] --> z
#z = x.iloc[np.where(x["mcq_questions_options"]=='[]')]

In [37]:
#z.head(1)

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
24572,96015345-0e43-48ee-84f7-88fae4aa1040,73470220-9f44-4207-a8e3-fd63bca12f2a,<p></p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:08:04,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,2018-08-30 14:12:56,talentio627,[],{},Capgemini QB 4


In [41]:
#z["subject"].isna().sum()

72859

In [38]:
#z["subject"].value_counts()

bulk-imported    27
Aptitude         20
Nandha            2
VISTAS            1
Name: subject, dtype: int64

In [40]:
#z.shape

(72909, 20)

In [47]:
#z.to_excel("ques_ans_empty.xlsx")

In [14]:
#mcq_single_correct_data.iloc[np.where(mcq_single_correct_data["question_data"] == '<p></p>')]

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
24572,96015345-0e43-48ee-84f7-88fae4aa1040,73470220-9f44-4207-a8e3-fd63bca12f2a,<p></p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:08:04,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,2018-08-30 14:12:56,talentio627,[],{},Capgemini QB 4
118766,dd0f3e0f-801b-44e8-8c1f-904aacf7ab28,8206727b-2f4b-4281-9847-d9b553be3c7b,<p></p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-31 13:49:17,6ff9556e-0409-48a2-b834-9f740f56a5e6,2018-09-04 09:04:21,NaT,talentio627,[],{},TAL-INF-M2-VA
128900,a31cdadf-0836-4112-91d8-04db08865ad0,184675e2-eaa8-4d39-8381-b4db5d911040,<p></p>,Medium,mcq_single_correct,original_question,a7411341-d28a-497e-b268-6ed9b949fffc,593ffd06-c76c-420d-913e-60ba2a30975f,4fa2a45a-8409-45ee-a5ac-af7da4e83aae,Nandha,End Semester,Unit_3,2020-12-18 03:55:42,6d8ac624-a6f3-4f1c-98cf-1259e6c1191b,2020-12-18 03:56:02,NaT,nandha416,[],{},Test bank 75q
134086,dd0f3e0f-801b-44e8-8c1f-904aacf7ab28,1390e91f-8246-4f0a-9d5a-61544fda07ca,<p></p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-31 13:49:17,6ff9556e-0409-48a2-b834-9f740f56a5e6,2018-09-04 09:03:12,NaT,talentio627,[],{},TAL-INF-M2-VA
160796,a31cdadf-0836-4112-91d8-04db08865ad0,87de919c-3a1f-41db-a81c-94f0b537e11c,<p></p>,Medium,mcq_single_correct,original_question,a7411341-d28a-497e-b268-6ed9b949fffc,593ffd06-c76c-420d-913e-60ba2a30975f,006c6627-6089-4ce7-8824-171974ea03d3,Nandha,End Semester,Unit_2,2020-12-18 03:55:42,6d8ac624-a6f3-4f1c-98cf-1259e6c1191b,2020-12-18 03:56:02,NaT,nandha416,[],{},Test bank 75q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8371567,82ed0a92-bb3f-4080-b0b7-2b70cd468fe0,d07e98d4-d61d-498f-9f2a-f3b7790312ee,<p></p>,Medium,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-12 13:24:19,77a55a51-0698-4d26-9396-08c05d689274,2020-12-12 13:44:28,NaT,vistas296,"[{""text"":""<p>programmable</p>"",""media"":""""},{""t...","{""args"":[""<p>programmable</p>""],""partial"":[]}",ESODD2020_18AMRO31_MANAGEMENT INFORMATION SYST...
8383738,82ed0a92-bb3f-4080-b0b7-2b70cd468fe0,ebb8bd43-bc38-4e32-90eb-09aabc3d613b,<p></p>,Medium,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-12 13:15:12,77a55a51-0698-4d26-9396-08c05d689274,2020-12-12 13:23:56,NaT,vistas296,"[{""text"":""<p>material flow</p>"",""media"":""""},{""...","{""args"":[""<p>All of the above</p>""],""partial"":[]}",ESODD2020_18AMRO31_MANAGEMENT INFORMATION SYST...
8386600,82ed0a92-bb3f-4080-b0b7-2b70cd468fe0,f6937637-2b71-47f5-915a-f6b3130f2944,<p></p>,Medium,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-12 13:15:11,77a55a51-0698-4d26-9396-08c05d689274,2020-12-12 13:23:56,NaT,vistas296,"[{""text"":""<p>higher returns on short-term inve...","{""args"":[""<p>All of the above</p>""],""partial"":[]}",ESODD2020_18AMRO31_MANAGEMENT INFORMATION SYST...
8392488,82ed0a92-bb3f-4080-b0b7-2b70cd468fe0,25623eeb-3779-4a2f-bb14-fe1b05d0923f,<p></p>,Medium,mcq_single_correct,original_question,e8a9b71a-8bdd-4208

## Taking the mcq_single_correct as whole data1 :

In [49]:
whole_data1 = mcq_single_correct_data

In [50]:
whole_data1.shape[0]

8493058

In [53]:
#whole_data_sample = whole_data1[::10]

In [57]:
del(whole_data_sample)

In [45]:
whole_data1.head(1)

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
0,bb56eb34-a6c5-4732-8eaa-5b8a1b9f5051,9aba1952-e8c7-40af-ab2c-1d8486585550,<p>Each class can inherit the base class _____...,Medium,mcq_single_correct,unlimited_purchase,9f91343b-be0e-4336-b199-30d823a58046,ddd3424f-0d3e-442e-9577-7a85e37c868c,f8c7d21b-28fd-4ff0-b1c9-ecdfc2abf51f,MCQ,Core JAVA,OOPS - Inheritance,2021-06-25 02:48:02,726462db-6397-4906-8793-df0406b5133a,NaT,NaT,neohire,"[{""text"":""<p>a) Independently using any inheri...","{""args"":[""<p>a) Independently using any inheri...",PREP_Java_PT_MCQ_Inheritance


In [55]:
# number of schools
whole_data_sample["school_code"].nunique()

215

#### Data Details:
Coumns:

|'qb_id'| 'q_id'| 'question_data'| 'manual_difficulty'| 'question_type'|'imported'| 'subject_id'| 'topic_id'| 

'sub_topic_id'|'subject'|'topic'| 'sub_topic'| 'createdAt'| 'school_id'| 'q_deletedAt'|'qb_deletedAt'| 'school_code'|

#### A coustoum class to perform cleaning, transformation, find duplicate questions and to show its variations

In [56]:
class duplicate_v1:
    def __init__(self,raw_data):
        self.data = raw_data
        
    def raw_data(self):
        raw_data = self.data
        return raw_data
    def col_to_drop(self,columns:list):
        return self.data.drop(columns,axis=1)
    def clean_data(self,data):
        data["question_data"] = data["question_data"].replace(r"<p></p>",np.NaN,regex=True)
        if not "clean_question_data" in data.columns:
            data.insert(loc=3,column="clean_question_data",value=data["question_data"].str.replace(r'<[^<>]*>','',regex=True).replace('&amp','').replace('&nbsp',''))
        #data["clean_question_data"] = data["question_data"].str.replace(r'<[^<>]*>','',regex=True)
        return data
    def clean_nan(self,data):
        data = data.drop(index=list(np.where(data["clean_question_data"] == "nan")[0]))
        return data
    def transform_data(self,cleaned_data):
        data_new = cleaned_data
        data_new["clean_question_data"] = [str(data_new["clean_question_data"].iloc[i]).strip().lower() for i in range(len(data_new["clean_question_data"]))]
        return data_new
    def filter_duplicate(self,transformed_data):
        data = transformed_data
        data["duplicate"] = data["clean_question_data"].duplicated()
        duplicated = data.iloc[np.where(data["duplicate"] == True)]
        return duplicated.reset_index()
    def find_dup_idx(self,data,scentence:str) -> list:
        return list(np.where(scentence == data["clean_question_data"][::1])[0])
    # to show the variations:
    def variations(self,filtered_data,idx:list):
        if len(filtered_data.iloc[idx].index)>1:
            print(f"filtered_data :: {len(filtered_data.iloc[idx].index)}")
            q_type = int(len(filtered_data["question_type"].iloc[idx]))
            if q_type > 2:
                print(f"q_type :: {q_type}")
                q_sub = int(len(filtered_data["subject"].iloc[idx]))
                if q_sub > 2:
                    print(f"q_sub :: {q_sub}")
                    q_topic = int(len(filtered_data["topic"].iloc[idx]))
                    if q_topic > 2:
                        print(f"q_topic :: {q_topic}")
                        q_sub_topic = int(len(filtered_data["sub_topic"].iloc[idx]))
                        if q_sub_topic > 2:
                            print(f"q_sub_topic :: {q_sub_topic}")
                            return filtered_data.iloc[idx]
        else:
            print("No variations found...")

#### Dropping the rows with empty questions...

In [58]:
# Empty question data rows
print(f"Number of questions in actual data :: {whole_data1.shape[0]}")
no_ques_data = whole_data1.iloc[np.where(whole_data1['question_data'] == "<p></p>")]
print(f"Number of rows containing empty question :: {no_ques_data.shape[0]}")
no_ques_data.head(2)

Number of questions in actual data :: 8493058
Number of rows containing empty question :: 0


,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name


In [59]:
whole_data1.shape

(8493058, 20)

In [52]:
# removed the rows with empty question data

cleaned_data1 = whole_data1.drop(index=no_ques_data.index,axis=0)

In [53]:
# sorting the index while storing the actual index values..
cleaned_data1 = cleaned_data1.reset_index()

In [54]:
cleaned_data1.shape

(8494372, 21)

In [55]:
# Carrying the actual data to processing:

duplicate = duplicate_v1(raw_data=cleaned_data1)

In [56]:
# removing the html tags from the data
cleaned_data2 = duplicate.clean_data(cleaned_data1)

In [57]:
cleaned_data2.shape

(8494372, 22)

In [58]:
cleaned_data2.head()

,index,qb_id,q_id,clean_question_data,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
0,0,bb56eb34-a6c5-4732-8eaa-5b8a1b9f5051,9aba1952-e8c7-40af-ab2c-1d8486585550,Each class can inherit the base class ________...,<p>Each class can inherit the base class _____...,Medium,mcq_single_correct,unlimited_purchase,9f91343b-be0e-4336-b199-30d823a58046,ddd3424f-0d3e-442e-9577-7a85e37c868c,...,Core JAVA,OOPS - Inheritance,2021-06-25 02:48:02,726462db-6397-4906-8793-df0406b5133a,NaT,NaT,neohire,"[{""text"":""<p>a) Independently using any inheri...","{""args"":[""<p>a) Independently using any inheri...",PREP_Java_PT_MCQ_Inheritance
1,1,851c2fdc-4723-43c1-993b-90b3f3d54ab7,1711c72e-5d12-4afd-964c-be91aa050573,"Select a process sequence, for machining the c...","<p><span style=""color: rgb(0, 0, 10);"">Select ...",Hard,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,03c113f6-ec06-424d-8d53-96be34075412,...,UNIT3,CO3,2021-10-26 11:18:45,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,NaT,psgtech850,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",NOV 21_19P501_COMPUTER NUMERICAL CONTROL MACHINES
2,3,e009bf4b-aa5b-43e5-804b-34272d1dbc0a,2fb0e331-abee-4b05-8509-08d11cd341e2,What will be the output of the given code?$$$e...,<p>What will be the output of the given code?<...,Easy,mcq_single_correct,unlimited_purchase,9f91343b-be0e-4336-b199-30d823a58046,1f66d012-a996-4d00-8f70-3a65ec3a634f,...,C Programming,Functions,2020-07-15 07:04:16,a334a0f7-2fce-465b-8216-1bb456f571f3,NaT,NaT,skct892,"[{""text"":""<p>2 3</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>2 3</p>""],""partial"":[]}",Accenture Technical Practice Test
3,4,32b25449-0e3e-4af4-b7a5-3ef0d25cb1f2,da4ecc6f-2393-4a07-b09f-71e139c65237,The range extended electric vehicle (REVs) con...,"<p><span style=""color: rgb(0, 0, 10);"">The ran...",Easy,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,4f7eb276-bbfa-446c-bcbc-d340a5559958,...,UNIT2,CO2,2021-10-21 08:34:12,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,2021-10-21 11:32:13,psgtech850,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",NOV21_19A505_ALTERNATE POWER TRAIN
4,5,a0dc8f1a-8a34-4b41-aaad-73b4f22c0582,e354f4c6-0c7d-4aa2-ae0f-25b39a65c59f,...,"<p><span style=""color: rgb(0, 0, 10);""> ...",Easy,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,43de6b0e-c7fe-45ca-9e42-f5cc9120b15f,...,UNIT1,CO1,2021-11-01 08:55:15,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,NaT,psgtech850,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",QB_20SA71_GAME THEORY_NOV 21


In [59]:
# remove the nan created while removing the html tags
cleaned_data3 = duplicate.clean_nan(cleaned_data2)
cleaned_data3.shape

(8494372, 22)

# Sampling the data

In [64]:
cleaned_data3_1st_sample = cleaned_data3[:8494372//3]

In [65]:
cleaned_data3_1st_sample.shape

(2831457, 22)

In [62]:
8494372//3

2831457

In [66]:
cleaned_data3_2nd_sample = cleaned_data3[2831457:(2831457*2)]
cleaned_data3_2nd_sample.shape

(2831457, 22)

In [67]:
cleaned_data3_3rd_sample = cleaned_data3[(2831457*2):]
cleaned_data3_3rd_sample.shape

(2831458, 22)

##

In [60]:
# Transforming data contains removing of unwanted spaces and converting the texts into lowercase to maintain uniformity in 
# comparison
transformed_data = duplicate.transform_data(cleaned_data3)

MemoryError: 

In [25]:
transformed_data.to_csv("TransformedData.csv")

In [27]:
transformed_data.isna().sum()

MemoryError: Unable to allocate 158. MiB for an array with shape (18, 9205964) and data type bool

In [22]:
# Filtering the questions whose duplicates exists in the entire data
filtered_data = duplicate.filter_duplicate(transformed_data)
print(f"Number of questions whose duplicates exists :: {filtered_data.shape[0]}")

MemoryError: Unable to allocate 1021. MiB for an array with shape (18, 7436948) and data type object

In [ ]:
# collecting the index of duplicates of a random single question
num = 41
idx = duplicate.find_dup_idx(filtered_data,cleaned_data3["clean_question_data"].iloc[num])
print(f"""Number of duplicate for question :: {cleaned_data3["clean_question_data"].iloc[num]}\n is {len(idx)}""")

In [ ]:
# fetching the variations of the above question
duplicate.variations(filtered_data=filtered_data,idx=idx)

In [ ]:
result = duplicate.variations(filtered_data=filtered_data,idx=idx)
result.to_excel("check.xlsx")

In [ ]:
duplicate.variations(filtered_data=filtered_data,idx=idx)["mcq_questions_options"].iloc[1]

In [ ]:
data = [{"text":"<p>5</p>","media":""},{"text":"<p>10</p>","media":""},{"text":"<p>Error</p>","media":""},{"text":"<p>1</p>","media":""}]

In [ ]:
d=data[3]

In [ ]:
list(d.values())[0].replace("<p>","").replace("</p>","")

In [ ]:
ans = []
for i in range(len(data)):
    d=data[i]
    ans.append(list(d.values())[0].replace("<p>","").replace("</p>",""))

In [ ]:
ans

In [ ]:
ops_null_mcq_s_c = transformed_data.iloc[np.where(transformed_data["mcq_questions_options"].iloc[np.where(transformed_data["question_type"] == "mcq_single_correct")].isnull() == True)]

In [ ]:
ops_null_mcq_s_c = ops_null_mcq_s_c.iloc[np.where(ops_null_mcq_s_c["mcq_questions_options"].isnull() == True)]

In [ ]:
ops_null_mcq_s_c.shape

In [ ]:
ops_null_mcq_s_c.columns

In [ ]:
ops_null_mcq_s_c.to_excel("noneData_mcq_s_c_1.xlsx",columns=['qb_id','q_id','clean_question_data','question_type','createdAt','school_id','mcq_questions_options', 'mcq_questions_answer','school_code','qb_name'])

In [ ]:
# from the sample data which school has the maximum number of empty option rows.  

ops_null_mcq_s_c["school_code"].value_counts()

In [ ]:
pd.DataFrame(ops_null_mcq_s_c["school_code"].value_counts()).plot(kind='pie',subplots=True)

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
palette_color = sns.color_palette('bright')
plt.pie(pd.DataFrame(ops_null_mcq_s_c["school_code"].value_counts()), labels=pd.DataFrame(ops_null_mcq_s_c["school_code"].values), colors=palette_color, autopct='%.0f%%')
plt.show()